In [ ]:
import numpy as np
import pytorch_lightning as pl
import torch
from src.data.functions import fetch_and_load_covtype_dataset
from src.models.model_surrogate_parallel import ModelSurrogateParallel
from torch.utils.data import DataLoader

# Loading data

In [ ]:
train_set, test_set = fetch_and_load_covtype_dataset()

train_loader = DataLoader(train_set, batch_size=128, shuffle=True)
test_loader = DataLoader(test_set, batch_size=128, shuffle=True)

# Training

In [ ]:
network = ModelSurrogateParallel(
    in_features=train_set.n_classes,
    out_features=train_set.n_features
)

trainer = pl.Trainer(max_epochs=10, accelerator='gpu')
trainer.fit(network, train_dataloaders=train_loader)

Notre modèle de substitution entrainé parallèlement aux predictions, arrivent à obtenir environ 71.58% sur le jeu d'entrainement et environ 71.86% sur le jeu de test, au niveau des prédictions similaires.

## Quel est le modèle le plus performant en exactitude ?

Sur les données de test, bien évidemment.

In [ ]:
test_targets = []
test_model_predictions = []
test_surrogate_predictions = []
for samples, tgts in test_loader:
    lbls_model, lbls_surrogate = network.predict_step(samples, None)
    test_model_predictions.append(lbls_model)
    test_surrogate_predictions.append(lbls_surrogate)
    test_targets.append(tgts)

test_targets = torch.hstack(test_targets)
test_model_predictions = torch.hstack(test_model_predictions)
test_surrogate_predictions = torch.hstack(test_surrogate_predictions)

(test_model_predictions == test_targets).sum() / len(test_set) * 100, (test_targets == test_surrogate_predictions).sum() / len(test_set) * 100, (test_model_predictions == test_surrogate_predictions).sum() / len(test_set) * 100

In [ ]:
train_targets = []
for _, tgts in train_loader:
    train_targets.append(tgts)

train_targets = []
train_model_predictions = []
train_surrogate_predictions = []
for samples, tgts in train_loader:
    lbls_model, lbls_surrogate = network.predict_step(samples, None)
    train_model_predictions.append(lbls_model)
    train_surrogate_predictions.append(lbls_surrogate)
    train_targets.append(tgts)

train_targets = torch.hstack(train_targets)
train_model_predictions = torch.hstack(train_model_predictions)
train_surrogate_predictions = torch.hstack(train_surrogate_predictions)

(train_model_predictions == train_targets).sum() / len(train_set) * 100, (train_targets == train_surrogate_predictions).sum() / len(train_set) * 100, (train_model_predictions == train_surrogate_predictions).sum() / len(train_set) * 100

Je pense qu'il reste plus optimal d'entrainer le modèle surrogate après l'entrainement du modèle de base.